In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import json
import pandas as pd
from pandas.io.json import json_normalize

In [0]:
import operator

def get_truth(relate):
    ops = {'>': operator.gt,
           '<': operator.lt,
           '>=': operator.ge,
           '<=': operator.le,
           '=': operator.eq}
    return ops[relate]

In [0]:
MainData = []
for line in open('netporterdata', 'r'):
    MainData.append(json.loads(line))

In [0]:
f_df = pd.DataFrame(MainData)

#1

In [0]:
df_price=json_normalize(data=f_df['price'])
df_1 = pd.concat([f_df['_id'],df_price], axis=1)

In [0]:
def make_discount(df,op,value):
  df['discount'] = 100*(df['regular_price.value'] - df['offer_price.value'])/df['regular_price.value']
  df_filtered = df[get_truth(op)(df.discount,value)]
  return df_filtered

In [0]:
discount_df = make_discount(df_1,'<',0.0)

#2

In [0]:
df_brand=json_normalize(data=f_df['brand'])
df_2 = pd.concat([f_df['_id'],df_price,df_brand], axis=1)

In [0]:
df_2['discount'] = 100*(df_2['regular_price.value'] - df_2['offer_price.value'])/df_2['regular_price.value']

In [0]:
df_3 = df_2.groupby('name') \
       .agg({'_id':'size','discount':'mean'}) \
       .rename(columns={'_id':'count','discount':'avg_discount'}) \
       .reset_index()

In [0]:
def get_count_discount(df,brand):
  t_count = df.loc[df.name == brand]['count'].tolist()[0]
  df[df.name == brand]['count']
  t_avg_discount = df.loc[df.name == brand]['avg_discount'].tolist()[0]
  return t_count,t_avg_discount

In [0]:
count,discount=get_count_discount(df_3,'gucci')

#3

In [0]:
df_brand=json_normalize(data=f_df['brand'])
df_price=json_normalize(data=f_df['price'])
df_5 = pd.concat([f_df['_id'],df_price,df_brand], axis=1)

In [0]:
c_price=[]

for i in range(f_df.shape[0]):
  try:
    c_price.append(f_df.similar_products[i]['website_results']['5da94f4e6d97010001f81d72']['knn_items'][0]['_source']['price']['basket_price']['value'])
  except IndexError:
    try:
      c_price.append(f_df.similar_products[i]['website_results']['5d0cc7b68a66a100014acdb0']['knn_items'][0]['_source']['price']['basket_price']['value'])
    except IndexError:
      try:
        c_price.append(f_df.similar_products[i]['website_results']['5da94e940ffeca000172b12a']['knn_items'][0]['_source']['price']['basket_price']['value'])
      except IndexError:
          try:
            c_price.append(f_df.similar_products[i]['website_results']['5da94ef80ffeca000172b12c']['knn_items'][0]['_source']['price']['basket_price']['value'])
          except IndexError:
            c_price.append(f_df.similar_products[i]['website_results']['5da94f270ffeca000172b12e']['knn_items'][0]['_source']['price']['basket_price']['value'])

df_5['c_price']=c_price

In [0]:
def competition_id (df,brand):
  if brand == "":
    return df.loc[df['basket_price.value'] > df.c_price]
  else:
    df = df.loc[df['basket_price.value'] > df.c_price]
    df = df.loc[df['name']==brand]
    return df

In [0]:
df_3f=competition_id (df_5,"balenciaga")

In [0]:
df_3f=competition_id (df_5,"")

#4

In [0]:
df_brand=json_normalize(data=f_df['brand'])
df_price=json_normalize(data=f_df['price'])
df_7 = pd.concat([f_df['_id'],f_df['website_id'],df_price,df_brand], axis=1)

In [0]:
farfetch_price = []

for i in range(f_df.shape[0]):
  try:
    farfetch_price.append(f_df.similar_products[i]['website_results']['5d0cc7b68a66a100014acdb0']['knn_items'][0]['_source']['price']['basket_price']['value'])
  except:
    farfetch_price.append(0.0)

In [0]:
for i in range(len(farfetch_price)):
  farfetch_price[i]=farfetch_price[i]+0.1*farfetch_price[i]

df_7['farfetch_price']=farfetch_price
df_7 = df_7.drop(df_7[df_7.farfetch_price == 0.0 ].index)
df_7 = df_7.loc[df_7['basket_price.value'] > df_7.farfetch_price]